In [2]:
###https://raw.githubusercontent.com/rasbt/python-machine-learning-book/master/code/datasets/wine/wine.data
#import pandas as pd 
#df_wine = pd.read_table('wine.txt', sep=",",header=None)

In [ ]:
###1. import the data
###pls find the data here: 
###https://raw.githubusercontent.com/rasbt/python-machine-learning-book/master/code/datasets/wine/wine.data
import pandas as pd 
df_wine = pd.read_table('wine.txt', sep=",",header=None)
 
df_wine.columns = ['Class label', 'Alcohol', 'Malic acid', 'Ash', 
'Alcalinity of ash', 'Magnesium', 'Total phenols', 
'Flavanoids', 'Nonflavanoid phenols', 'Proanthocyanins', 
'Color intensity', 'Hue', 'OD280/OD315 of diluted wines', 'Proline']
 
###2. Observe 
print(df_wine.head())
###SKIPPED
 
###3. Preprocess
from sklearn.model_selection import train_test_split
X, y = df_wine.iloc [:, 1:].values, df_wine.iloc[:, 0].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
 
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt 
 
lr0 = LogisticRegression()
lr0 = lr0.fit(X_train, y_train)
print(lr0.score(X_test,y_test))
###0.944444444444
 
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train_std = sc.fit_transform(X_train)
X_test_std = sc.transform(X_test)
 
###We apply LDA for X_train with y_train
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
lda = LDA(n_components=2)
X_train_lda = lda.fit_transform(X_train_std, y_train)
lr = LogisticRegression()
lr = lr.fit(X_train_lda, y_train)
 
###Here we define a function to visualize the decision boundaries
 
from matplotlib.colors import ListedColormap
import numpy as np 
 
def plot_decision_regions(X, y, classifier, resolution=0.02):
 
    # setup marker generator and color map
    markers = ('s', 'x', 'o', '^', 'v')
    colors = ('red', 'blue', 'lightgreen', 'gray', 'cyan')
    cmap = ListedColormap(colors[:len(np.unique(y))])
 
    # plot the decision surface
    x1_min, x1_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    x2_min, x2_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx1, xx2 = np.meshgrid(np.arange(x1_min, x1_max, resolution),
                         np.arange(x2_min, x2_max, resolution))
    Z = classifier.predict(np.array([xx1.ravel(), xx2.ravel()]).T)
    Z = Z.reshape(xx1.shape)
    plt.contourf(xx1, xx2, Z, alpha=0.4, cmap=cmap)
    plt.xlim(xx1.min(), xx1.max())
    plt.ylim(xx2.min(), xx2.max())
 
    # plot class samples
    for idx, cl in enumerate(np.unique(y)):
        plt.scatter(x=X[y == cl, 0], y=X[y == cl, 1],
                    alpha=0.8, c=cmap(idx),
                    marker=markers[idx], label=cl)
 
plot_decision_regions(X_train_lda, y_train, classifier=lr)
plt.xlabel('LD 1')
plt.ylabel('LD 2')
plt.legend(loc='lower left')
plt.tight_layout()
plt.show()

In [ ]:
###We apply LDA for X_test with y_test
X_test_lda = lda.transform(X_test_std)
plot_decision_regions(X_test_lda, y_test, classifier=lr)
plt.xlabel('LD 1')
plt.ylabel('LD 2')
plt.legend(loc='lower left')
plt.tight_layout()
# plt.savefig('./images/lda4.png', dpi=300)
plt.show()
 
print(lr.score(X_test_lda,y_test))
###1.0 Overfitting might be the problem here

In [ ]:
###If we are dealing with nonlinear problems, which we may encounter rather frequently 
###in real-world applications, linear transformation techniques for dimensionality 
###reduction, such as PCA and LDA, may not be the best choice. 
###Using kernel PCA, we can transform data that is not linearly separable onto a new, 
###lower-dimensional subspace that is suitable for linear classiers.
 
########################################################################
############   We build the kernel PCA from zero ground     ############
########################################################################
 
from scipy.spatial.distance import pdist, squareform
from scipy import exp
from scipy.linalg import eigh
import numpy as np
 
def rbf_kernel_pca(X, gamma, n_components):
    """
    RBF kernel PCA implementation.
 
    Parameters
    ------------
    X: {NumPy ndarray}, shape = [n_samples, n_features]
         
    gamma: float
      Tuning parameter of the RBF kernel
         
    n_components: int
      Number of principal components to return
 
    Returns
    ------------
     X_pc: {NumPy ndarray}, shape = [n_samples, k_features]
       Projected dataset   
 
    """
    # Calculate pairwise squared Euclidean distances
    # in the MxN dimensional dataset.
    sq_dists = pdist(X, 'sqeuclidean')
 
    # Convert pairwise distances into a square matrix.
    # Kernel trick 
    mat_sq_dists = squareform(sq_dists)
 
    # Compute the symmetric kernel matrix.
    K = exp(-gamma * mat_sq_dists)
 
    # Center the kernel matrix.
    N = K.shape[0]
    one_n = np.ones((N,N)) / N
    K = K - one_n.dot(K) - K.dot(one_n) + one_n.dot(K).dot(one_n)
 
    # Obtaining eigenpairs from the centered kernel matrix
    # numpy.eigh returns them in sorted order
    eigvals, eigvecs = eigh(K)
 
    # Collect the top k eigenvectors (projected samples)
    X_pc = np.column_stack((eigvecs[:, -i]
                            for i in range(1, n_components + 1)))
 
    return X_pc
 
from sklearn.datasets import make_circles
from sklearn.decomposition import PCA
from sklearn.datasets import make_moons
 
X, y = make_circles(n_samples=1000, random_state=123, noise=0.1, factor=0.2)
 
plt.scatter(X[y==0, 0], X[y==0, 1], color='red', marker='^', alpha=0.5)
plt.scatter(X[y==1, 0], X[y==1, 1], color='blue', marker='o', alpha=0.5)
 
plt.tight_layout()
plt.show()

In [ ]:
###We do it with PCA
scikit_pca = PCA(n_components=2)
X_spca = scikit_pca.fit_transform(X)
 
fig, ax = plt.subplots(nrows=1,ncols=2, figsize=(7,3))
 
ax[0].scatter(X_spca[y==0, 0], X_spca[y==0, 1], 
            color='red', marker='^', alpha=0.5)
ax[0].scatter(X_spca[y==1, 0], X_spca[y==1, 1],
            color='blue', marker='o', alpha=0.5)
 
ax[1].scatter(X_spca[y==0, 0], np.zeros((500,1))+0.02, 
            color='red', marker='^', alpha=0.5)
ax[1].scatter(X_spca[y==1, 0], np.zeros((500,1))-0.02,
            color='blue', marker='o', alpha=0.5)
 
ax[0].set_xlabel('PC1')
ax[0].set_ylabel('PC2')
ax[1].set_ylim([-1, 1])
ax[1].set_yticks([])
ax[1].set_xlabel('PC1')
 
plt.tight_layout()
plt.show()

In [ ]:
###We do it with Kenel PCA
X_kpca = rbf_kernel_pca(X, gamma=15, n_components=2)
 
fig, ax = plt.subplots(nrows=1,ncols=2, figsize=(7,3))
ax[0].scatter(X_kpca[y==0, 0], X_kpca[y==0, 1], 
            color='red', marker='^', alpha=0.5)
ax[0].scatter(X_kpca[y==1, 0], X_kpca[y==1, 1],
            color='blue', marker='o', alpha=0.5)
 
ax[1].scatter(X_kpca[y==0, 0], np.zeros((500,1))+0.02, 
            color='red', marker='^', alpha=0.5)
ax[1].scatter(X_kpca[y==1, 0], np.zeros((500,1))-0.02,
            color='blue', marker='o', alpha=0.5)
 
ax[0].set_xlabel('PC1')
ax[0].set_ylabel('PC2')
ax[1].set_ylim([-1, 1])
ax[1].set_yticks([])
ax[1].set_xlabel('PC1')
 
plt.tight_layout()
# plt.savefig('./figures/circles_3.png', dpi=300)
plt.show()

In [ ]:
########################################################################
############       We build the kernel PCA from sklearn     ############
########################################################################
from sklearn.decomposition import KernelPCA
 
X, y = make_moons(n_samples=100, random_state=123)
scikit_kpca = KernelPCA(n_components=2, kernel='rbf', gamma=15)
X_skernpca = scikit_kpca.fit_transform(X)
 
plt.scatter(X_skernpca[y==0, 0], X_skernpca[y==0, 1], 
            color='red', marker='^', alpha=0.5)
plt.scatter(X_skernpca[y==1, 0], X_skernpca[y==1, 1], 
            color='blue', marker='o', alpha=0.5)
 
plt.xlabel('PC1')
plt.ylabel('PC2')
plt.tight_layout()
# plt.savefig('./figures/scikit_kpca.png', dpi=300)
plt.show()